<a href="https://colab.research.google.com/github/macapagithub/2022-docker/blob/main/2.2.Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Modelos de clasificación de texto

En este notebook se muestra cómo crear un modelo de clasificación de texto.

Empezamos instalando la libreria [datasets](https://huggingface.co/docs/datasets/index) y [transformers](https://huggingface.co/docs/transformers/index) de huggingface.

In [1]:
!pip install transformers[torch] datasets -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

## Librerias:

* AutoTokenizer: Carga automáticamente un tokenizador pre-entrenado para un modelo específico.
* AutoModelForSequenceClassification: Carga automáticamente un modelo pre-entrenado para la clasificación de secuencias.
* Trainer: Clase para entrenar y evaluar modelos de Transformers.
* TrainingArguments: Define los argumentos para el entrenamiento del modelo.

* datasets: para cargar datasets preprocesados.
  * La función load_dataset permite cargar datasets preprocesados desde Hugging Face Hub.

In [2]:
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


Descargar y preparar el dataset:

Cargamos el dataset de emociones "emotion" y lo dividimos en DataFrames de pandas para entrenamiento, validación y prueba.

In [3]:
emotion = load_dataset("emotion")

train_df = emotion["train"].to_pandas()
valid_df = emotion["validation"].to_pandas()
test_df = emotion["test"].to_pandas()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Cargamos el tokenizador asociado al modelo pre-entrenado.

In [4]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True)
valid_encodings = tokenizer(valid_df['text'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

El tokenizador convierte las palabras en sub-unidades más pequeñas (tokens) que el modelo puede procesar.
Los parámetros truncation y padding garantizan que todas las entradas tengan la misma longitud, lo cual es necesario para el entrenamiento del modelo.

In [5]:
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EmotionDataset(train_encodings, train_df['label'].tolist())
valid_dataset = EmotionDataset(valid_encodings, valid_df['label'].tolist())
test_dataset = EmotionDataset(test_encodings, test_df['label'].tolist())


In [40]:
for n in train_dataset:
  print(n)
  break

{'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

TrainingArguments para configurar los parámetros de entrenamiento.

* output_dir='./results': Directorio donde se guardarán los resultados del entrenamiento.
* num_train_epochs=3: Número de épocas de entrenamiento (3 en este caso).
* per_device_train_batch_size=16: Tamaño del batch de entrenamiento por dispositivo (GPU o CPU).
* per_device_eval_batch_size=64: Tamaño del batch de evaluación por dispositivo (puede ser mayor que el de entrenamiento).
* warmup_steps=500: Número de pasos de calentamiento(warm-up steps) para aumentar gradualmente el learning rate.
* weight_decay=0.01: Coeficiente de regularización L2 para evitar el sobreajuste.
* logging_dir='./logs': Directorio para almacenar los logs del entrenamiento.
* logging_steps=10: Frecuencia con la que se registran métricas en los logs (cada 10 pasos).

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.782100
20,1.779800
30,1.771200
40,1.731300
50,1.710800
60,1.684400
70,1.636100
80,1.564000
90,1.553000
100,1.572800


TrainOutput(global_step=3000, training_loss=0.27703798371553423, metrics={'train_runtime': 451.8778, 'train_samples_per_second': 106.223, 'train_steps_per_second': 6.639, 'total_flos': 1080514292544000.0, 'train_loss': 0.27703798371553423, 'epoch': 3.0})

In [7]:
results = trainer.evaluate(test_dataset)
print("Accuracy:", results['eval_accuracy'])

Accuracy: 0.9295


Evaluar el modelo en el conjunto de prueba

In [8]:
results = trainer.evaluate(test_dataset)
print("Accuracy:", results['eval_accuracy'])


Accuracy: 0.9295


Realizar predicciones:

In [34]:
model.device.type

'cuda'

In [38]:
text = "I can go from feeling so hopeless to so damn hopeful just to be around someone who cares about me."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
inputs.to(model.device.type)
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=1)
print(f"Predicted class: {predictions.item()}")


Predicted class: 0
